In [1]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
import time

In [2]:
df = pd.read_csv('sementes.csv')
df

,semente,dominio,robotstxt,localpath,htmlname
0,https://www.srlupa.com.br,srlupa.com.br,False,pages/sementes/srlupa/,srlupa
1,https://serviceira.com,serviceira.com,False,pages/sementes/serviceira/,serviceira
2,https://conectu.com.br,conectu.com.br,False,pages/sementes/conectu/,conectu
3,https://www.achouservicos.com.br,achouservicos.com.br,False,pages/sementes/achouservicos/,achouservicos
4,https://www.telelistas.net/br/clinicas+de+psic...,telelistas.net,True,pages/sementes/telelistas/clinicasdepsicologia/,clinicasdepsicologia
5,https://www.telelistas.net/br/clinicas+psiquia...,telelistas.net,True,pages/sementes/telelistas/clinicaspsiquiatricas/,clinicaspsiquiatricas
6,https://www.telelistas.net/br/psicanalistas,telelistas.net,True,pages/sementes/telelistas/psicanalistas/,psicanalistas
7,https://www.telelistas.net/br/psicologos,telelistas.net,True,pages/sementes/telelistas/psicologos/,psicologos
8,https://www.telelistas.net/br/psicopedagogos,telelistas.net,True,pages/sementes/telelistas/psicopedagogos/,psicopedagogos
9,https://freelancer.com.br,freelancer.com.br,True,pages/sementes/freelancer/,freelancer


In [3]:
def getDisallows(path):
    ret_list = []
    file = open(path+"robots.txt", "r")
    lines = file.readlines()
    read = False
    for line in lines:
        if(line.endswith("\n")):
            line = line[:-1]
        if(line.lower().startswith("user-agent:") and line.endswith("*")):
            read = True
        elif(line.lower().startswith("user-agent:") and not(line.endswith("*"))):
            read = False
        elif(line.lower().startswith("disallow:") and read):
            aux = line[10:]
            
            if(aux.endswith("*")):
                aux = aux[:-1]
            if(aux.endswith("?")):
                aux = aux[:-1]
            if(aux.endswith("?")):
                aux = aux[:-1]
            if(aux.endswith("*")):
                aux = aux[:-1]
                
            ret_list.append(aux)
    return ret_list

In [ ]:
domain_block_list = []
link_accessed_list = []
link_to_access_list = []

for i in range(len(df)):
    #iterando na semente
    domain = df['dominio'][i]
    
    print("Crawleando a semente:\t"+domain)
    
    link_accessed_list.append(df['semente'][i])
    print("Domínio da semente adicionado a lista de links acessados")
    
    path = df['localpath'][i]
    
    if(df['robotstxt'][i]):
        domain_block_list = getDisallows(path)
        print("Restrições detectadas no Robots.txt:\t"+domain_block_list)
        
        
    url = path+df['htmlname'][i]+".html"
    parsed_page = BeautifulSoup(open(url,encoding="latin1"),"html.parser")
    
    print("Capturando links para a fronteira")
    total_links = parsed_page.find_all('a')
    print("Total de links na página:\t"+str(len(total_links)))
    
    lenght_front_count_aux = len(link_to_access_list)
    
    for link in total_links:
        if(domain in link.get('href')):
            link_to_access_list.append(link.get('href'))
        elif(link.get('href').startswith("/")):
            link_to_access_list.append(domain+link.get('href'))
            
    lenght_front_count_aux = len(link_to_access_list) - lenght_front_count_aux
    
    print("Total de links adicionados a fronteira:\t"+str(lenght_front_count_aux))
    
    time_to_sleep = 0
    print("Aguardando "+str(time_to_sleep)+" segundos para a próxima iteração")
    time.sleep(time_to_sleep)
    
    for link in link_to_access_list:
        if(not link.startswith("http://") and not link.startswith("https://")):
            link = "http://"+link
        
        block = False
        for blockitem in domain_block_list:
            if(blockitem in link):
                block = True
                break
        if(not block and link not in link_accessed_list):
            try:
                print("Tentando acessar "+link+"   ...")
                ret=rq.get(link)
                
                
                #TODO DOWNLOAD DA PÁGINA COM CLASSIFICADOR
                
                
                link_accessed_list.append(df['semente'][i])
                parsed_page = BeautifulSoup(ret.text,"html.parser")


                print("Capturando links para a fronteira")
                total_links = parsed_page.find_all('a')
                print("Total de links na página:\t"+str(len(total_links)))

                lenght_front_count_aux = len(link_to_access_list)

                for linkItem in total_links:
                    if(domain in linkItem.get('href')):
                        link_to_access_list.append(linkItem.get('href'))
                    elif(linkItem.get('href').startswith("/")):
                        link_to_access_list.append(domain+linkItem.get('href'))

                lenght_front_count_aux = len(link_to_access_list) - lenght_front_count_aux

                print("Total de links adicionados a fronteira:\t"+str(lenght_front_count_aux))
                print("Tamanho atual da fronteira:\t"+str(len(link_to_access_list)))
                print("Crawler concluiu sua passada por "+str(link)+"   ...")
            except:
                print("Erro ao acessar o link "+str(link)+"\nO Crawler continuará do próximo na fronteira")
    print("Crawler concluiu atividade na semente "+domain+"   ...")
    print("\n---------------------\n----\n---------------------\n")
print("Crawler finalizou sua atividade")

Crawleando a semente:	srlupa.com.br
Domínio da semente adicionado a lista de links acessados
Capturando links para a fronteira
Total de links na página:	72
Total de links adicionados a fronteira:	48
Aguardando 0 segundos para a próxima iteração
Tentando acessar http://srlupa.com.br/   ...
Capturando links para a fronteira
Total de links na página:	72
Total de links adicionados a fronteira:	48
Tamanho atual da fronteira:	96
Crawler concluiu sua passada por http://srlupa.com.br/   ...
Tentando acessar http://srlupa.com.br/accounts/login/   ...
Capturando links para a fronteira
Total de links na página:	46
Total de links adicionados a fronteira:	32
Tamanho atual da fronteira:	128
Crawler concluiu sua passada por http://srlupa.com.br/accounts/login/   ...
Tentando acessar http://srlupa.com.br/ofereca-seus-servicos/   ...
Capturando links para a fronteira
Total de links na página:	12
Total de links adicionados a fronteira:	9
Tamanho atual da fronteira:	137
Crawler concluiu sua passada por h

Capturando links para a fronteira
Total de links na página:	58
Total de links adicionados a fronteira:	49
Tamanho atual da fronteira:	5584
Crawler concluiu sua passada por https://blog.srlupa.com.br/7-reparos-de-casa-que-voce-sempre-precisa-fazer   ...
Tentando acessar https://blog.srlupa.com.br/7-reparos-de-casa-que-voce-sempre-precisa-fazer   ...
Capturando links para a fronteira
Total de links na página:	58
Total de links adicionados a fronteira:	49
Tamanho atual da fronteira:	5633
Crawler concluiu sua passada por https://blog.srlupa.com.br/7-reparos-de-casa-que-voce-sempre-precisa-fazer   ...
Tentando acessar https://blog.srlupa.com.br/7-reparos-de-casa-que-voce-sempre-precisa-fazer   ...
Capturando links para a fronteira
Total de links na página:	58
Total de links adicionados a fronteira:	49
Tamanho atual da fronteira:	5682
Crawler concluiu sua passada por https://blog.srlupa.com.br/7-reparos-de-casa-que-voce-sempre-precisa-fazer   ...
Tentando acessar http://srlupa.com.br/sobre/ 

Capturando links para a fronteira
Total de links na página:	12
Total de links adicionados a fronteira:	9
Tamanho atual da fronteira:	7121
Crawler concluiu sua passada por http://srlupa.com.br/ofereca-seus-servicos/   ...
Tentando acessar http://srlupa.com.br/categorias/#category-9   ...
Capturando links para a fronteira
Total de links na página:	424
Total de links adicionados a fronteira:	385
Tamanho atual da fronteira:	7506
Crawler concluiu sua passada por http://srlupa.com.br/categorias/#category-9   ...
Tentando acessar http://srlupa.com.br/categorias/#category-10   ...
Capturando links para a fronteira
Total de links na página:	424
Total de links adicionados a fronteira:	385
Tamanho atual da fronteira:	7891
Crawler concluiu sua passada por http://srlupa.com.br/categorias/#category-10   ...
Tentando acessar http://srlupa.com.br/categorias/#category-12   ...
Capturando links para a fronteira
Total de links na página:	424
Total de links adicionados a fronteira:	385
Tamanho atual da fr

Capturando links para a fronteira
Total de links na página:	48
Erro ao acessar o link http://srlupa.com.br/cadastro-sr-lupa/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/eventos/fotografia/   ...
Capturando links para a fronteira
Total de links na página:	197
Erro ao acessar o link http://srlupa.com.br/profissionais/eventos/fotografia/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/tecnologia/analista-de-ti/   ...
Capturando links para a fronteira
Total de links na página:	277
Erro ao acessar o link http://srlupa.com.br/profissionais/tecnologia/analista-de-ti/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/construcao-e-reformas/arquiteto/   ...
Capturando links para a fronteira
Total de links na página:	312
Erro ao acessar o link http://srlupa.com.br/profissionais/construcao-e-reformas/arquiteto/
O Crawler continuará do próximo na front

Capturando links para a fronteira
Total de links na página:	49
Total de links adicionados a fronteira:	35
Tamanho atual da fronteira:	13847
Crawler concluiu sua passada por http://srlupa.com.br/empresa/termos-e-condicoes-de-uso/   ...
Tentando acessar http://srlupa.com.br/central-de-ajuda/perguntas-frequentes/   ...
Capturando links para a fronteira
Total de links na página:	77
Total de links adicionados a fronteira:	36
Tamanho atual da fronteira:	13883
Crawler concluiu sua passada por http://srlupa.com.br/central-de-ajuda/perguntas-frequentes/   ...
Tentando acessar http://srlupa.com.br/central-de-ajuda/   ...
Capturando links para a fronteira
Total de links na página:	55
Total de links adicionados a fronteira:	36
Tamanho atual da fronteira:	13919
Crawler concluiu sua passada por http://srlupa.com.br/central-de-ajuda/   ...
Tentando acessar http://srlupa.com.br/cadastro-sr-lupa/   ...
Capturando links para a fronteira
Total de links na página:	48
Erro ao acessar o link http://srlupa.c

...

Capturando links para a fronteira
Total de links na página:	96
Erro ao acessar o link http://srlupa.com.br/profissionais/eventos/produtor/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/eventos/promotor/   ...
Capturando links para a fronteira
Total de links na página:	107
Erro ao acessar o link http://srlupa.com.br/profissionais/eventos/promotor/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/eventos/recepcionista/   ...
Capturando links para a fronteira
Total de links na página:	182
Erro ao acessar o link http://srlupa.com.br/profissionais/eventos/recepcionista/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/eventos/seguranca/   ...
Capturando links para a fronteira
Total de links na página:	108
Erro ao acessar o link http://srlupa.com.br/profissionais/eventos/seguranca/
O Crawler continuará do próximo na fronteira
Tentando acessar http

Capturando links para a fronteira
Total de links na página:	44
Total de links adicionados a fronteira:	29
Tamanho atual da fronteira:	20394
Crawler concluiu sua passada por http://srlupa.com.br/profissionais/saude-bem-estar/obstetra/   ...
Tentando acessar http://srlupa.com.br/profissionais/saude-bem-estar/pilates/   ...
Capturando links para a fronteira
Total de links na página:	48
Erro ao acessar o link http://srlupa.com.br/profissionais/saude-bem-estar/pilates/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/saude-bem-estar/psicologo/   ...
Capturando links para a fronteira
Total de links na página:	110
Erro ao acessar o link http://srlupa.com.br/profissionais/saude-bem-estar/psicologo/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/saude-bem-estar/quiropratico-massagem/   ...
Capturando links para a fronteira
Total de links na página:	84
Erro ao acessar o link http://srlupa.com.br/pro

Capturando links para a fronteira
Total de links na página:	64
Erro ao acessar o link http://srlupa.com.br/profissionais/servicos-domesticos/redes-de-protecao/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/servicos-domesticos/transporte-escolar/   ...
Capturando links para a fronteira
Total de links na página:	47
Erro ao acessar o link http://srlupa.com.br/profissionais/servicos-domesticos/transporte-escolar/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/servicos-domesticos/transporte-particular/   ...
Capturando links para a fronteira
Total de links na página:	104
Erro ao acessar o link http://srlupa.com.br/profissionais/servicos-domesticos/transporte-particular/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/servicos-domesticos/veterinario/   ...
Capturando links para a fronteira
Total de links na página:	51
Erro ao acessar o link htt

Capturando links para a fronteira
Total de links na página:	1425
Erro ao acessar o link http://srlupa.com.br/profissionais/construcao-e-reformas/eletricista/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/construcao-e-reformas/pedreiro/   ...
Capturando links para a fronteira
Total de links na página:	816
Erro ao acessar o link http://srlupa.com.br/profissionais/construcao-e-reformas/pedreiro/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/construcao-e-reformas/pintor/   ...
Capturando links para a fronteira
Total de links na página:	1162
Erro ao acessar o link http://srlupa.com.br/profissionais/construcao-e-reformas/pintor/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/servicos-domesticos/diarista/   ...
Capturando links para a fronteira
Total de links na página:	296
Erro ao acessar o link http://srlupa.com.br/profissionais/servicos-dom

Capturando links para a fronteira
Total de links na página:	55
Total de links adicionados a fronteira:	36
Tamanho atual da fronteira:	22922
Crawler concluiu sua passada por http://srlupa.com.br/central-de-ajuda/   ...
Tentando acessar http://srlupa.com.br/cadastro-sr-lupa/   ...
Capturando links para a fronteira
Total de links na página:	48
Erro ao acessar o link http://srlupa.com.br/cadastro-sr-lupa/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/eventos/fotografia/   ...
Capturando links para a fronteira
Total de links na página:	197
Erro ao acessar o link http://srlupa.com.br/profissionais/eventos/fotografia/
O Crawler continuará do próximo na fronteira
Tentando acessar http://srlupa.com.br/profissionais/tecnologia/analista-de-ti/   ...
Capturando links para a fronteira
Total de links na página:	277
Erro ao acessar o link http://srlupa.com.br/profissionais/tecnologia/analista-de-ti/
O Crawler continuará do próximo na fronteira
Tentan